In [ ]:
from sklearn.externals import joblib
import re
import requests
import pandas as pd

In [ ]:
#cd '/home/jovyan/pkl'

In [ ]:
tfidf_vec = joblib.load('/home/jovyan/pkl/tfidf_vec.pkl')
prediction_model = joblib.load('/home/jovyan/pkl/prediction_model.pkl')

In [ ]:
def cleaner(text):
    text = re.sub('[A-Z]\\n',' ', text)
    text = re.sub('{.*}', '', text)
    text = re.sub('\s\s', ' ', text)
    text = re.sub('[^a-zA-Z ]',' ',text)
    text = re.sub('\\t', ' ', text)
    text = re.sub('\s\s', ' ', text)
    return text

In [ ]:
title = extract_title_from_url('https://en.wikipedia.org/wiki/Epos_Now')
title

In [ ]:
def generate_page_query(url):
    '''
    Format an api call for requests
    '''
    query = """http://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&explaintext=True&titles={}""".format(url.split('/')[-1])
    return query

In [ ]:
query = 'https://en.wikipedia.org/wiki/Epos_Now'
response_url = requests.get(generate_page_query(query))

In [ ]:
response_url

In [ ]:
page_id = list(response_url.json()['query']['pages'].keys())[0]
if int(page_id) == -1:
    print ('Invalid URL given or wikipedia page not found')
else:
    page_text = response_url.json()['query']['pages'][page_id]['extract']
cleaned_text = cleaner(page_text)
cleaned_text = [cleaned_text]
type(cleaned_text)
X = tfidf_vec.transform(cleaned_text)
predicted_category = prediction_model.predict(X)[0]
probability = list(zip(prediction_model.classes_, prediction_model.predict_proba(X)[0]))

proba_dict = {}
for key, val in enumerate(prediction_model.classes_):
    print(val)
    print(prediction_model.predict_proba(X)[0][key])
    proba_dict[val] = prediction_model.predict_proba(X)[0][key]
proba_dict

In [ ]:
predicted_category[0]

In [ ]:
probability

## Only the code block below is needed to go into the python file.

In [1]:
from sklearn.externals import joblib
from collections import defaultdict
import re
import requests
import pandas as pd
tfidf_vec = joblib.load('/home/jovyan/pkl/tfidf_vec.pkl')
prediction_model = joblib.load('/home/jovyan/pkl/prediction_model.pkl')

def cleaner(text):
    text = re.sub('[A-Z]\\n',' ', text)
    text = re.sub('{.*}', '', text)
    text = re.sub('\s\s', ' ', text)
    text = re.sub('[^a-zA-Z ]',' ',text)
    text = re.sub('\\t', ' ', text)
    text = re.sub('\s\s', ' ', text)
    return text

def generate_page_query(url):
    '''
    Format an api call for requests
    '''
    query = """http://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&explaintext=True&titles={}""".format(url.split('/')[-1])
    return query

def make_page_prediction(url):
    response_url = requests.get(generate_page_query(url))
    page_id = list(response_url.json()['query']['pages'].keys())[0]
    
    if int(page_id) == -1:
        print ('Invalid URL given or wikipedia page not found')
    else:
        page_text = response_url.json()['query']['pages'][page_id]['extract']
        #cleaned_text = cleaner(page_text)
        #cleaned_text = [cleaned_text]
        #type(cleaned_text)
        X = tfidf_vec.transform([cleaner(page_text)])
        predicted_category = prediction_model.predict(X)[0]   
        proba_dict = {}
        for key, val in enumerate(prediction_model.classes_):
            proba_dict[val] = prediction_model.predict_proba(X)[0][key]
    return predicted_category, proba_dict[predicted_category]

In [2]:
url = 'https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine'
cat, proba = make_page_prediction(url)
print('Page: {} \nPredicted Category: {} \nProbability: {}'.format(url, cat, proba))


Page: https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine 
Predicted Category: machine_learning 
Probability: 0.9998701554385739
